<a href="https://colab.research.google.com/github/thiagocarvalho93/wunderground_webscraping/blob/main/Wunderground_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Bibliotecas
# Instalação das bibliotecas necessárias
!pip install --upgrade gspread
# Importa os packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
import time
import re
from dateutil.relativedelta import relativedelta

In [32]:
#@title Função: Estação e data -> Dataframe do Pandas - Diário
link_base = 'https://www.wunderground.com/dashboard/pws/'

def to_dataframe(estacao, date):
  url= link_base + estacao + '/table/' + date + '/' + date  + '/daily'
  # Cria uma variável contendo o conteúdo da página
  page = requests.get(url)
  # Interpreta os dados com o BeautifulSoup
  soup = BeautifulSoup(page.text, 'html.parser')

  # Procura a tabela correta no html
  table = soup.find('table',class_='history-table desktop-table')
  # Usa o Pandas pra ler a tabela html
  df = pd.read_html(str(table))[0]
  df = df.dropna(how='all')

  # Tira as strings das unidades para converter para numérico
  df2 = df.replace({'°F': ''}, regex=True)
  df2 = df2.replace({'°%': ''}, regex=True)
  df2 = df2.replace({'°mph': ''}, regex=True)
  df2 = df2.replace({'°in': ''}, regex=True)
  df2 = df2.replace({'w/m²': ''}, regex=True)
  cols = df2.columns.drop(['Time','Wind','UV']) # Colunas numéricas
  # Tira os espaços em branco
  for col in cols:
    df2[col]  = df2[col].str.strip()
  # Converte para numérico
  df2[cols] = df2[cols].apply(pd.to_numeric)

  # Farenheit para Celsius
  f = lambda x: (x - 32) * 5/9
  df2[['Temperature',	'Dew Point']] = df2[['Temperature',	'Dew Point']].applymap(f).round(1)
  # mph para km/h:
  g = lambda x: x* 1.60934
  df2[['Speed','Gust']] = df2[['Speed','Gust']].applymap(g).round(1)
  # Identifica a estação na tabela
  df2['Estação'] = estacao

  # Converte o horário para o formato ISO8601
  date = datetime.strptime(date, "%Y-%m-%d")
  x = pd.to_datetime(df2['Time']).apply(lambda dt: dt.replace(day=date.day, month=date.month, year=date.year))
  df2['ISO8601'] = x.dt.strftime('%Y-%m-%dT%H:%M:%SZ')

  # Preenche os valores NaN com 0 
  # (p/ evitar erro na hora de exportar para o sheets)
  df2 = df2.fillna(0)

  return df2

In [33]:
#@title Aquisição dos dados

# Definindo a data de ontem
# Fuso horário
timezone_offset = -3.0
tzinfo = timezone(timedelta(hours = timezone_offset))
# Ontem no formato usado no link:
ontem = (datetime.now(tzinfo) - relativedelta(days=1)).strftime("%Y-%m-%d")

df = pd.DataFrame()

# Definindo as estações
#@markdown Insira aqui o código das estações
stations = ['IMACA7', 'IMACA13', 'ICAMPO96', 'IMACA15', 'IRIODA1','IRJSANTA2']
# Definindo a data
date = ontem #@markdown Insira a data desejada aqui - no formato AAAA-MM-DD


# Aplicando as funções e coletando os dados 
for station in stations:
  print(station)
  df_temp = to_dataframe(station,date)
  df = df.append(df_temp)

# Converte as colunas selecionadas em Float

print(df)


IMACA7
IMACA13
ICAMPO96
IMACA15
IRIODA1
IRJSANTA2
         Time  Temperature  Dew Point  ...  Solar    Estação               ISO8601
1    12:04 AM         25.5       23.7  ...    0.0     IMACA7  2021-12-15T00:04:00Z
2    12:09 AM         25.5       23.8  ...    0.0     IMACA7  2021-12-15T00:09:00Z
3    12:14 AM         25.5       24.0  ...    0.0     IMACA7  2021-12-15T00:14:00Z
4    12:19 AM         25.4       23.9  ...    0.0     IMACA7  2021-12-15T00:19:00Z
5    12:24 AM         25.4       23.9  ...    0.0     IMACA7  2021-12-15T00:24:00Z
..        ...          ...        ...  ...    ...        ...                   ...
284  11:39 PM         27.2       22.8  ...    0.0  IRJSANTA2  2021-12-15T23:39:00Z
285  11:44 PM         27.3       22.8  ...    0.0  IRJSANTA2  2021-12-15T23:44:00Z
286  11:49 PM         27.3       22.8  ...    0.0  IRJSANTA2  2021-12-15T23:49:00Z
287  11:54 PM         27.2       22.9  ...    0.0  IRJSANTA2  2021-12-15T23:54:00Z
288  11:59 PM         27.2       22.8

In [35]:
#@title Passar dados para o Google Sheets
# Credenciamento
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Abre a planilha e atualiza os dados
worksheet = gc.open('Dados Meteorológicos Macaé').sheet1
worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1kNvvyPPyiXJcGYKgNfjlRgAhB-IsE74TrEzpsPDB9Lg',
 'updatedCells': 21994,
 'updatedColumns': 14,
 'updatedRange': "'Página1'!A1:N1571",
 'updatedRows': 1571}